<a href="https://colab.research.google.com/github/kridtapon/WFO-TrixEFI-Flow/blob/main/WFO_TrixEFI_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00


In [21]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Trix Indicator (Triple Exponential Moving Average)
def calculate_trix(df, period=14):
    """
    Calculate the Trix Indicator.
    """
    # First, calculate the Exponential Moving Averages (EMA)
    ema1 = df['Close'].ewm(span=period).mean()
    ema2 = ema1.ewm(span=period).mean()
    ema3 = ema2.ewm(span=period).mean()

    # Trix is the rate of change of the third EMA
    trix = ema3.pct_change() * 100  # Multiply by 100 to express it as a percentage

    return trix

# Function to calculate Elder's Force Index (EFI)
def calculate_efi(df, period=13):
    """
    Calculate Elder's Force Index (EFI).
    """
    efi = (df['Close'] - df['Close'].shift(1)) * df['Volume']
    efi = efi.rolling(window=period).mean()  # Smoothing over the specified period
    return efi

# Define the stock symbol and time period
symbol = 'NFLX'  # APO NFLX PLTR
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with Trix and EFI
def walk_forward_optimization_trix_efi(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    trix_period_range = range(5, 31)  # Range for Trix periods
    efi_period_range = range(5, 31)   # Range for EFI periods

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of Trix and EFI parameters
        for params in itertools.product(trix_period_range, efi_period_range):
            trix_period, efi_period = params

            # Calculate indicators on the training data
            train_data['Trix'] = calculate_trix(train_data, trix_period)
            train_data['EFI'] = calculate_efi(train_data, efi_period)

            # Generate entry and exit signals
            entries = (train_data['Trix'] > 0) & (train_data['EFI'] > 0)  # Both Trix and EFI positive for entry
            exits = (train_data['Trix'] < 0) & (train_data['EFI'] < 0)    # Both Trix and EFI negative for exit

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (trix_period, efi_period)

        # Test with the best parameters on the test data
        test_data['Trix'] = calculate_trix(test_data, best_params[0])
        test_data['EFI'] = calculate_efi(test_data, best_params[1])

        entries = (test_data['Trix'] > 0) & (test_data['EFI'] > 0)
        exits = (test_data['Trix'] < 0) & (test_data['EFI'] < 0)

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)


# Perform walk-forward optimization
results = walk_forward_optimization_trix_efi(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply Trix and EFI indicators
    yearly_data['Trix'] = calculate_trix(yearly_data, params[0])
    yearly_data['EFI'] = calculate_efi(yearly_data, params[1])

    # Define entry/exit conditions
    entries = (yearly_data['Trix'] > 0) & (yearly_data['EFI'] > 0)
    exits = (yearly_data['Trix'] < 0) & (yearly_data['EFI'] < 0)

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-21-4642b209fb4d>:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-4642b209fb4d>:66: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-4642b209fb4d>:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide


Walk-Forward Optimization Results:
   Year Best_Params
0  2020    (30, 16)
1  2021     (29, 5)
2  2022     (20, 6)
3  2023     (20, 6)
4  2024     (25, 6)
5  2025     (25, 6)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                            429147.95201
Total Return [%]                       329.147952
Benchmark Return [%]                   170.252574
Max Gross Exposure [%]                      100.0
Total Fees Paid                       2619.680241
Max Drawdown [%]                        28.259102
Max Drawdown Duration                       536.0
Total Trades                                    8
Total Closed Trades                             7
Total Open Trades                               1
Open Trade PnL                      120300.850989
Win Rate [%]                            71.428571
Best Trade [%]          

<ipython-input-21-4642b209fb4d>:126: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-4642b209fb4d>:127: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-4642b209fb4d>:126: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-4

In [22]:
# Filter Test Years
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,270252.573883
Total Return [%],170.252574
Benchmark Return [%],170.252574
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],75.947318
